In [35]:
from nnsight import NNsight

In [36]:
from transformers import GPTJForCausalLM, AutoTokenizer  
model_path = r"C:\Users\allan\ResearchStuff\checkpoint-1953"

gptj = GPTJForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [37]:
test = NNsight(gptj)

In [38]:
text = "GPTJ"

input_ids = tokenizer(text, return_tensors="pt").input_ids

gen_tokens = gptj.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [39]:
with test.trace(gen_tokens) as tracer:
    h0_output = test.transformer.h[0].output.save()
    h1_output = test.transformer.h[1].output.save()

print(h0_output)
print(h1_output)

(tensor([[[ -0.6713,  -1.8850,   0.1562,  ...,  -0.4049,  -1.4577,  -1.6844],
         [ -1.0820,  -1.5449,   0.8616,  ...,   2.4094,  -1.6196,   0.0721],
         [  0.4574,  -3.6470,   0.2879,  ...,   2.5716,   1.5716,   0.9055],
         ...,
         [ 10.3733,  -8.0781,   3.3568,  ...,  17.3844,  -9.1563,  -0.8096],
         [-15.1537, -19.1792,  -3.2441,  ...,  27.9173,  11.6917,   4.5771],
         [-24.8388, -45.5067, -15.0219,  ...,  52.7458,   2.0810,  -5.5088]]],
       grad_fn=<AddBackward0>), DynamicCache())
(tensor([[[ 1.7886e+00, -1.4395e+01, -1.2068e+00,  ...,  1.4213e+00,
           1.2189e+00,  9.3767e+00],
         [-1.8669e-01, -9.9831e+00,  5.2910e-01,  ...,  6.7367e+00,
          -7.6761e-01,  7.6162e+00],
         [-2.4369e+00, -6.1992e+00,  5.6117e-01,  ...,  5.5680e+00,
          -1.8104e+00,  1.2986e+00],
         ...,
         [ 8.1073e+00, -1.7238e+01,  8.3116e+00,  ...,  2.5976e+01,
          -6.6501e+00,  8.7413e+00],
         [-1.9196e+01, -2.0085e+01,  1

In [49]:
outputs = []
with test.trace(gen_tokens) as tracer:
    for i in range(12):
        outputs.append((f"Layer {i} output", test.transformer.h[i].output.save()))

In [50]:
for layer in outputs: 
    print(layer)

('Layer 0 output', (tensor([[[ -0.6713,  -1.8850,   0.1562,  ...,  -0.4049,  -1.4577,  -1.6844],
         [ -1.0820,  -1.5449,   0.8616,  ...,   2.4094,  -1.6196,   0.0721],
         [  0.4574,  -3.6470,   0.2879,  ...,   2.5716,   1.5716,   0.9055],
         ...,
         [ 10.3733,  -8.0781,   3.3568,  ...,  17.3844,  -9.1563,  -0.8096],
         [-15.1537, -19.1792,  -3.2441,  ...,  27.9173,  11.6917,   4.5771],
         [-24.8388, -45.5067, -15.0219,  ...,  52.7458,   2.0810,  -5.5088]]],
       grad_fn=<AddBackward0>), DynamicCache()))
('Layer 1 output', (tensor([[[ 1.7886e+00, -1.4395e+01, -1.2068e+00,  ...,  1.4213e+00,
           1.2189e+00,  9.3767e+00],
         [-1.8669e-01, -9.9831e+00,  5.2910e-01,  ...,  6.7367e+00,
          -7.6761e-01,  7.6162e+00],
         [-2.4369e+00, -6.1992e+00,  5.6117e-01,  ...,  5.5680e+00,
          -1.8104e+00,  1.2986e+00],
         ...,
         [ 8.1073e+00, -1.7238e+01,  8.3116e+00,  ...,  2.5976e+01,
          -6.6501e+00,  8.7413e+00],

In [67]:
import torch 
"""output = test.transformer.h[i].output.save()
        print(i)
        print(output)
        print(output.type)
        outputs.append((f"Layer {i} output", test.transformer.h[i].output.save()))  """

outputs = []    
probs= []
with test.trace(gen_tokens) as tracer:
    for i in range(12):
        outputs.append((f"Layer {i} output", test.transformer.h[i].output.save()))
        probs.append(test.transformer.h[i].output.save())

In [76]:
import torch 
new_prob = []

for prob in probs:
    new = torch.nn.functional.softmax(prob[0], dim=-1)
    new_prob.append(new)
